In [3]:
import ollama
from sentence_transformers import SentenceTransformer
import numpy as np
from qdrant_client import QdrantClient
from qdrant_client.http import models
from faker import Faker

In [9]:

embedding_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# sample data
fake = Faker()
payload = [{"song": fake.sentence(), "artist": fake.name(), "year": fake.year()} for _ in range(1000)]

# embeddings
embeddings = []
for item in payload:
    text = f"{item['artist']} {item['song']}"
    embedding = embedding_model.encode(text)
    embeddings.append(embedding)

embeddings = np.array(embeddings)

# quadrant client
client = QdrantClient(host="localhost", port=6333)

my_collection = "song_collection"

# check if collection exists, if not, create it
try:
    client.get_collection(collection_name=my_collection)
    print(f"Collection '{my_collection}' already exists. Skipping creation.")
except Exception as e:
    print(f"Collection '{my_collection}' does not exist. Creating a new one.")
    client.create_collection(
        collection_name=my_collection,
        vectors_config=models.VectorParams(size=768, distance=models.Distance.COSINE)  
    )

# upsert data (embedding vectors and payload) into the Qdrant collection
index = list(range(len(payload)))
client.upsert(
    collection_name=my_collection,
    points=models.Batch(
        ids=index,
        vectors=embeddings.tolist(),
        payloads=payload
    )
)

# semantic search using a query
query = "What song was released in 1975?"
query_embedding = embedding_model.encode(query)

search_result = client.search(
    collection_name=my_collection,
    query_vector=query_embedding.tolist(),
    limit=10  # limit the results to top 10 similar items
)

print("Search Results from Qdrant:")
for hit in search_result:
    print(f"ID: {hit.id}, Score: {hit.score}, Payload: {hit.payload}")

# Mistral 7B via Ollama for a query-based response
response = ollama.chat(model="mistral", messages=[{"role": "user", "content": query}])

print("\nResponse from Mistral 7B (via Ollama):")
print(response["message"]["content"])

Collection 'song_collection' already exists. Skipping creation.


/var/folders/pv/dgx586t95xn1cl6sps79kygh0000gn/T/ipykernel_8066/1567350088.py:47: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = client.search(


Search Results from Qdrant:
ID: 704, Score: 0.43843365, Payload: {'song': 'On three onto few wind.', 'artist': 'Peter Olson', 'year': '2010'}
ID: 563, Score: 0.40363032, Payload: {'song': 'Choice chance beautiful so song yourself.', 'artist': 'Richard Phillips', 'year': '2012'}
ID: 58, Score: 0.40332446, Payload: {'song': 'Cover foreign voice.', 'artist': 'Mary Schwartz', 'year': '2006'}
ID: 152, Score: 0.39829558, Payload: {'song': 'Part unit song among culture return family trial.', 'artist': 'Cristian Baker', 'year': '1973'}
ID: 331, Score: 0.39770675, Payload: {'song': 'Day the black budget eight one.', 'artist': 'Mary Bryant', 'year': '1982'}
ID: 820, Score: 0.387292, Payload: {'song': 'Black fine center decade.', 'artist': 'James Taylor', 'year': '1980'}
ID: 302, Score: 0.383424, Payload: {'song': 'Think shake out foot them.', 'artist': 'Karen Mcbride', 'year': '2008'}
ID: 805, Score: 0.3725108, Payload: {'song': 'Record state film any off.', 'artist': 'Robert Moon', 'year': '199